# Overview

This notebook plots empirical results on UKBB predictive modeling, reanalyzing & further exploring previous work reported in Dadi et al 2021, GigaScience.


In [1]:
import pathlib
import numpy as np
import pandas as pd
import altair as alt

## Prepare data

In [2]:
from collections import OrderedDict
"""Number of UKBB variables
"""
from collections import OrderedDict

primary_demographics = OrderedDict([
    ('31-0.0', 'Gender'),
    ('21003-2.0', ' Age when attended assessment centre'),
    ('34-0.0', 'Year of birth'),
    ('52-0.0', 'Month of birth'),
    ('21022-0.0', 'Age at recruitment'),
])

lifestyle = OrderedDict([
    ('670-2.0', 'Type of accomodation lived in'),
    ('680-2.0', 'Own or rent accomodation lived in'),
    ('6139-2.0', 'Gas or solid-fuel cooking/heating'),
    ('699-2.0', 'Length of time at current address'),
    ('709-2.0', 'Number in household'),
    ('6141-2.0', 'How are people in household related to participant'),
    ('728-2.0', 'Number of vehicles in household'),
    ('738-2.0', 'Income before tax'),
    ('796-2.0', 'Distance between home and job workplace'),
    ('757-2.0', 'Time employed in main current job'),
    ('767-2.0', 'Length of working week for main job'),
    ('777-2.0', 'Freq. of travelling from home to job workplace'),
    ('6143-2.0', 'Transport type for commuting to job workplace'),
    ('6142-2.0', 'Current employment status'),
    ('806-2.0', 'Job involves mainly walking or standing'),
    ('816-2.0', 'Job involves heavy manual or physical work'),
    ('826-2.0', 'Job involves shift work'),
    ('3426-2.0', 'Job involves night shift work'),
    ('1031-2.0', 'Freq. of friend/ family visits'),
    ('6160-2.0', 'Leisure/social activities'),
    ('2110-2.0', 'Able to confide'),
    ('1239-2.0', 'Current tobacco smoking'),
    ('1249-2.0', 'Past tobacco smoking'),
    ('1259-2.0', 'Smoking/smokers in household'),
    ('1269-2.0', 'Exposure to tobacco smoke at home'),
    ('1279-2.0', 'Exposure to tobacco smoke outside home'),
    ('2644-2.0', 'Light smokers, at least 100 smokes in lifetime'),
    ('2867-2.0', 'Age started smoking in former smokers'),
    ('2877-2.0', 'Type of tobacco previously smoked'),
    ('2887-2.0', 'Number of cigarettes previously smoked daily'),
    ('2897-2.0', 'Age stopped smoking'),
    ('2907-2.0', 'Ever stopped smoking for 6+ months'),
    ('2926-2.0', 'Number of unsuccessful stop-smoking attempts'),
    ('2936-2.0', 'Likelihood of resuming smoking'),
    ('3426-2.0', 'Job involve night shift work'),
    ('3436-2.0', 'Age started smoking in current smokers'),
    ('3446-2.0', 'Type of tobacco currently smoked'),
    ('3456-2.0', 'Number of cigarettes currently smoked daily (current cigarette smokers)'),
    ('3466-2.0', 'Time from waking to first cigarette'),
    ('3476-2.0', 'Difficulty not smoking for 1 day'),
    ('3486-2.0', 'Ever tried to stop smoking'),
    ('3496-2.0', 'Wants to stop smoking'),
    ('3506-2.0', 'Smoking compared to 10 years previous'),
    ('5959-2.0',  'Previously smoked cigarettes on most/all days'),
    ('6157-2.0',  'Why stopped smoking'),
    ('6158-2.0',  'Why reduced smoking'),
])

education = OrderedDict([
    ('6138-2.0', 'Qualifications'),
    ('845-2.0', 'Age completed full time education')
])

earlylife = OrderedDict([
    ('1647-2.0', 'Country of birth (UK/elsewhere)'),
    ('1677-2.0', 'Breastfed as a baby'),
    ('1687-2.0', 'Comparitive body size at age 10'),
    ('1697-2.0', 'Comparitive height size at age 10'),
    ('1707-2.0', 'Handedness (chirality/laterality)'),
    ('1767-2.0', 'Adopted as a child'),
    ('1777-2.0', 'Part of a multiple birth'),
    ('1787-2.0', 'Maternal smoking around birth')
])

mental_health = OrderedDict([
    ('2040-2.0', 'Risk taking'),
    ('4526-2.0', 'Happiness'),
    ('4537-2.0', 'Work/job satisfaction'),
    ('4548-2.0', 'Health satisfaction'),
    ('4559-2.0', 'Family relationship satisfaction'),
    ('4570-2.0', 'Friendships satisfaction'),
    ('4581-2.0', 'Financial situation satisfaction'),
    ('4598-2.0', 'Ever depressed for a whole week'),
    ('4609-2.0', 'Longest period of depression'),
    ('4620-2.0', 'Number of depression episodes'),
    ('4631-2.0', 'Ever unenthusiastic/disinterested for a whole week'),
    ('4642-2.0', 'Ever manic/hyper for 2 days'),
    ('4653-2.0', 'Ever highly irritable/argumentative for 2 days'),
    ('2050-2.0', 'Frequency of depressed mood in last 2 weeks'),
    ('2060-2.0', 'Frequency of unenthusiasm / disinterest in last 2 weeks'),
    ('2070-2.0', 'Frequency of tenseness / restlessness in last 2 weeks'),
    ('2080-2.0', 'Frequency of tiredness / lethargy in last 2 weeks'),
    ('2090-2.0', 'Seen doctor (GP) for nerves, anxiety, tension or depression'),
    ('2100-1.0', 'Seen a psychiatrist for nerves, anxiety, tension or depression'),
    ('5375-2.0', 'Longest period of unenthusiasm / disinterest'),
    ('5386-2.0', 'Number of unenthusiastic/disinterested episodes'),
    ('5663-2.0', 'Length of longest manic/irritable episode'),
    ('5674-2.0', 'Severity of manic/irritable episode'),
    ('6145-2.0', 'Illness, injury, bereavement, stress in last 2 years'),
    ('6156-2.0', 'Manic/hyper symptoms')
])

fluid_intelligence = OrderedDict([
    ('20016-2.0', 'Fluid intelligence score')
])

neuroticism = OrderedDict([
    ('20127-0.0', 'Neuroticism score')
])

brain_smri = OrderedDict([
    ('25005-2.0', 'Volume of grey matter (normalised)'),
    # ('25006-2.0', 'Volume of grey matter'),
    ('25009-2.0', 'Volume of brain, grey+white matter (normalised)'),
    # ('25010-2.0', 'Volume of brain, grey+white matter'),
    ('25011-2.0', 'Volume of thalamus (left)'),
    ('25012-2.0', 'Volume of thalamus (right)'),
    ('25013-2.0', 'Volume of caudate (left)'),
    ('25014-2.0', 'Volume of caudate (right)'),
    ('25015-2.0', 'Volume of putamen (left)'),
    ('25016-2.0', 'Volume of putamen (right)'),
    ('25017-2.0', 'Volume of pallidum (left)'),
    ('25018-2.0', 'Volume of pallidum (right)'),
    ('25019-2.0', 'Volume of hippocampus (left)'),
    ('25020-2.0', 'Volume of hippocampus (right)'),
    ('25021-2.0', 'Volume of amygdala (left)'),
    ('25022-2.0', 'Volume of amygdala (right)'),
    ('25023-2.0', 'Volume of accumbens (left)'),
    ('25024-2.0', 'Volume of accumbens (right)'),
    ('25025-2.0', 'Volume of brain stem + 4th ventricle)'),
])

brain_smri_plus = OrderedDict([
    ('25001-2.0', 'Volume of peripheral cortical grey matter (normalised for head size)'),
    #('25002-2.0', 'Volume of peripheral cortical grey matter'),
    #('25003-2.0', 'Volume of ventricular cerebrospinal fluid (normalised for head size)'),
    #('25004-2.0', 'Volume of ventricular cerebrospinal fluid'),
    ('25005-2.0', 'Volume of grey matter (normalised)'),
    #('25006-2.0', 'Volume of grey matter'),
    #('25007-2.0', 'Volume of white matter (normalised for head size)'),
    #('25008-2.0', 'Volume of white matter'),
    ('25009-2.0', 'Volume of brain, grey+white matter (normalised)'),
    #('25010-2.0', 'Volume of brain, grey+white matter'),
    ('25011-2.0', 'Volume of thalamus (left)'),
    ('25012-2.0', 'Volume of thalamus (right)'),
    ('25013-2.0', 'Volume of caudate (left)'),
    ('25014-2.0', 'Volume of caudate (right)'),
    ('25015-2.0', 'Volume of putamen (left)'),
    ('25016-2.0', 'Volume of putamen (right)'),
    ('25017-2.0', 'Volume of pallidum (left)'),
    ('25018-2.0', 'Volume of pallidum (right)'),
    ('25019-2.0', 'Volume of hippocampus (left)'),
    ('25020-2.0', 'Volume of hippocampus (right)'),
    ('25021-2.0', 'Volume of amygdala (left)'),
    ('25022-2.0', 'Volume of amygdala (right)'),
    ('25023-2.0', 'Volume of accumbens (left)'),
    ('25024-2.0', 'Volume of accumbens (right)'),
    ('25025-2.0', 'Volume of brain stem + 4th ventricle)'),
    ('25782-2.0', 'Volume of grey matter in Frontal Pole (left)'),
    ('25783-2.0', 'Volume of grey matter in Frontal Pole (right)'),
    ('25784-2.0', 'Volume of grey matter in Insular Cortex (left)'),
    ('25785-2.0', 'Volume of grey matter in Insular Cortex (right)'),
    ('25786-2.0', 'Volume of grey matter in Superior Frontal Gyrus (left)'),
    ('25787-2.0', 'Volume of grey matter in Superior Frontal Gyrus (right)'),
    ('25788-2.0', 'Volume of grey matter in Middle Frontal Gyrus (left)'),
    ('25789-2.0', 'Volume of grey matter in Middle Frontal Gyrus (right)'),
    ('25790-2.0', 'Volume of grey matter in Inferior Frontal Gyrus, pars triangularis (left)'),
    ('25791-2.0', 'Volume of grey matter in Inferior Frontal Gyrus, pars triangularis (right)'),
    ('25792-2.0', 'Volume of grey matter in Inferior Frontal Gyrus, pars opercularis (left)'),
    ('25793-2.0', 'Volume of grey matter in Inferior Frontal Gyrus, pars opercularis (right)'),
    ('25794-2.0', 'Volume of grey matter in Precentral Gyrus (left)'),
    ('25795-2.0', 'Volume of grey matter in Precentral Gyrus (right)'),
    ('25796-2.0', 'Volume of grey matter in Temporal Pole (left)'),
    ('25797-2.0', 'Volume of grey matter in Temporal Pole (right)'),
    ('25798-2.0', 'Volume of grey matter in Superior Temporal Gyrus, anterior division (left)'),
    ('25799-2.0', 'Volume of grey matter in Superior Temporal Gyrus, anterior division (right)'),
    ('25800-2.0', 'Volume of grey matter in Superior Temporal Gyrus, posterior division (left)'),
    ('25801-2.0', 'Volume of grey matter in Superior Temporal Gyrus, posterior division (right)'),
    ('25802-2.0', 'Volume of grey matter in Middle Temporal Gyrus, anterior division (left)'),
    ('25803-2.0', 'Volume of grey matter in Middle Temporal Gyrus, anterior division (right)'),
    ('25804-2.0', 'Volume of grey matter in Middle Temporal Gyrus, posterior division (left)'),
    ('25805-2.0', 'Volume of grey matter in Middle Temporal Gyrus, posterior division (right)'),
    ('25806-2.0', 'Volume of grey matter in Middle Temporal Gyrus, temporooccipital part (left)'),
    ('25807-2.0', 'Volume of grey matter in Middle Temporal Gyrus, temporooccipital part (right)'),
    ('25808-2.0', 'Volume of grey matter in Inferior Temporal Gyrus, anterior division (left)'),
    ('25809-2.0', 'Volume of grey matter in Inferior Temporal Gyrus, anterior division (right)'),
    ('25810-2.0', 'Volume of grey matter in Inferior Temporal Gyrus, posterior division (left)'),
    ('25811-2.0', 'Volume of grey matter in Inferior Temporal Gyrus, posterior division (right)'),
    ('25812-2.0', 'Volume of grey matter in Inferior Temporal Gyrus, temporooccipital part (left)'),
    ('25813-2.0', 'Volume of grey matter in Inferior Temporal Gyrus, temporooccipital part (right)'),
    ('25814-2.0', 'Volume of grey matter in Postcentral Gyrus (left)'),
    ('25815-2.0', 'Volume of grey matter in Postcentral Gyrus (right)'),
    ('25816-2.0', 'Volume of grey matter in Superior Parietal Lobule (left)'),
    ('25817-2.0', 'Volume of grey matter in Superior Parietal Lobule (right)'),
    ('25818-2.0', 'Volume of grey matter in Supramarginal Gyrus, anterior division (left)'),
    ('25819-2.0', 'Volume of grey matter in Supramarginal Gyrus, anterior division (right)'),
    ('25820-2.0', 'Volume of grey matter in Supramarginal Gyrus, posterior division (left)'),
    ('25821-2.0', 'Volume of grey matter in Supramarginal Gyrus, posterior division (right)'),
    ('25822-2.0', 'Volume of grey matter in Angular Gyrus (left)'),
    ('25823-2.0', 'Volume of grey matter in Angular Gyrus (right)'),
    ('25824-2.0', 'Volume of grey matter in Lateral Occipital Cortex, superior division (left)'),
    ('25825-2.0', 'Volume of grey matter in Lateral Occipital Cortex, superior division (right)'),
    ('25826-2.0', 'Volume of grey matter in Lateral Occipital Cortex, inferior division (left)'),
    ('25827-2.0', 'Volume of grey matter in Lateral Occipital Cortex, inferior division (right)'),
    ('25828-2.0', 'Volume of grey matter in Intracalcarine Cortex (left)'),
    ('25829-2.0', 'Volume of grey matter in Intracalcarine Cortex (right)'),
    ('25830-2.0', 'Volume of grey matter in Frontal Medial Cortex (left)'),
    ('25831-2.0', 'Volume of grey matter in Frontal Medial Cortex (right)'),
    ('25832-2.0', 'Volume of grey matter in Juxtapositional Lobule Cortex (formerly Supplementary Motor Cortex) (left)'),
    ('25833-2.0', 'Volume of grey matter in Juxtapositional Lobule Cortex (formerly Supplementary Motor Cortex) (right)'),
    ('25834-2.0', 'Volume of grey matter in Subcallosal Cortex (left)'),
    ('25835-2.0', 'Volume of grey matter in Subcallosal Cortex (right)'),
    ('25836-2.0', 'Volume of grey matter in Paracingulate Gyrus (left)'),
    ('25837-2.0', 'Volume of grey matter in Paracingulate Gyrus (right)'),
    ('25838-2.0', 'Volume of grey matter in Cingulate Gyrus, anterior division (left)'),
    ('25839-2.0', 'Volume of grey matter in Cingulate Gyrus, anterior division (right)'),
    ('25840-2.0', 'Volume of grey matter in Cingulate Gyrus, posterior division (left)'),
    ('25841-2.0', 'Volume of grey matter in Cingulate Gyrus, posterior division (right)'),
    ('25842-2.0', 'Volume of grey matter in Precuneous Cortex (left)'),
    ('25843-2.0', 'Volume of grey matter in Precuneous Cortex (right)'),
    ('25844-2.0', 'Volume of grey matter in Cuneal Cortex (left)'),
    ('25845-2.0', 'Volume of grey matter in Cuneal Cortex (right)'),
    ('25846-2.0', 'Volume of grey matter in Frontal Orbital Cortex (left)'),
    ('25847-2.0', 'Volume of grey matter in Frontal Orbital Cortex (right)'),
    ('25848-2.0', 'Volume of grey matter in Parahippocampal Gyrus, anterior division (left)'),
    ('25849-2.0', 'Volume of grey matter in Parahippocampal Gyrus, anterior division (right)'),
    ('25850-2.0', 'Volume of grey matter in Parahippocampal Gyrus, posterior division (left)'),
    ('25851-2.0', 'Volume of grey matter in Parahippocampal Gyrus, posterior division (right)'),
    ('25852-2.0', 'Volume of grey matter in Lingual Gyrus (left)'),
    ('25853-2.0', 'Volume of grey matter in Lingual Gyrus (right)'),
    ('25854-2.0', 'Volume of grey matter in Temporal Fusiform Cortex, anterior division (left)'),
    ('25855-2.0', 'Volume of grey matter in Temporal Fusiform Cortex, anterior division (right)'),
    ('25856-2.0', 'Volume of grey matter in Temporal Fusiform Cortex, posterior division (left)'),
    ('25857-2.0', 'Volume of grey matter in Temporal Fusiform Cortex, posterior division (right)'),
    ('25858-2.0', 'Volume of grey matter in Temporal Occipital Fusiform Cortex (left)'),
    ('25859-2.0', 'Volume of grey matter in Temporal Occipital Fusiform Cortex (right)'),
    ('25860-2.0', 'Volume of grey matter in Occipital Fusiform Gyrus (left)'),
    ('25861-2.0', 'Volume of grey matter in Occipital Fusiform Gyrus (right)'),
    ('25862-2.0', 'Volume of grey matter in Frontal Operculum Cortex (left)'),
    ('25863-2.0', 'Volume of grey matter in Frontal Operculum Cortex (right)'),
    ('25864-2.0', 'Volume of grey matter in Central Opercular Cortex (left)'),
    ('25865-2.0', 'Volume of grey matter in Central Opercular Cortex (right)'),
    ('25866-2.0', 'Volume of grey matter in Parietal Operculum Cortex (left)'),
    ('25867-2.0', 'Volume of grey matter in Parietal Operculum Cortex (right)'),
    ('25868-2.0', 'Volume of grey matter in Planum Polare (left)'),
    ('25869-2.0', 'Volume of grey matter in Planum Polare (right)'),
    ('25870-2.0', 'Volume of grey matter in Heschls Gyrus (includes H1 and H2) (left)'),
    ('25871-2.0', 'Volume of grey matter in Heschls Gyrus (includes H1 and H2) (right)'),
    ('25872-2.0', 'Volume of grey matter in Planum Temporale (left)'),
    ('25873-2.0', 'Volume of grey matter in Planum Temporale (right)'),
    ('25874-2.0', 'Volume of grey matter in Supracalcarine Cortex (left)'),
    ('25875-2.0', 'Volume of grey matter in Supracalcarine Cortex (right)'),
    ('25876-2.0', 'Volume of grey matter in Occipital Pole (left)'),
    ('25877-2.0', 'Volume of grey matter in Occipital Pole (right)'),
    ('25878-2.0', 'Volume of grey matter in Thalamus (left)'),
    ('25879-2.0', 'Volume of grey matter in Thalamus (right)'),
    ('25880-2.0', 'Volume of grey matter in Caudate (left)'),
    ('25881-2.0', 'Volume of grey matter in Caudate (right)'),
    ('25882-2.0', 'Volume of grey matter in Putamen (left)'),
    ('25883-2.0', 'Volume of grey matter in Putamen (right)'),
    ('25884-2.0', 'Volume of grey matter in Pallidum (left)'),
    ('25885-2.0', 'Volume of grey matter in Pallidum (right)'),
    ('25886-2.0', 'Volume of grey matter in Hippocampus (left)'),
    ('25887-2.0', 'Volume of grey matter in Hippocampus (right)'),
    ('25888-2.0', 'Volume of grey matter in Amygdala (left)'),
    ('25889-2.0', 'Volume of grey matter in Amygdala (right)'),
    ('25890-2.0', 'Volume of grey matter in Ventral Striatum (left)'),
    ('25891-2.0', 'Volume of grey matter in Ventral Striatum (right)'),
    ('25892-2.0', 'Volume of grey matter in Brain-Stem'),
    ('25893-2.0', 'Volume of grey matter in I-IV Cerebellum (left)'),
    ('25894-2.0', 'Volume of grey matter in I-IV Cerebellum (right)'),
    ('25895-2.0', 'Volume of grey matter in V Cerebellum (left)'),
    ('25896-2.0', 'Volume of grey matter in V Cerebellum (right)'),
    ('25897-2.0', 'Volume of grey matter in VI Cerebellum (left)'),
    ('25898-2.0', 'Volume of grey matter in Vermis VI Cerebellum'),
    ('25899-2.0', 'Volume of grey matter in VI Cerebellum (right)'),
    ('25900-2.0', 'Volume of grey matter in Crus I Cerebellum (left)'),
    ('25901-2.0', 'Volume of grey matter in Vermis Crus I Cerebellum'),
    ('25902-2.0', 'Volume of grey matter in Crus I Cerebellum (right)'),
    ('25903-2.0', 'Volume of grey matter in Crus II Cerebellum (left)'),
    ('25904-2.0', 'Volume of grey matter in Vermis Crus II Cerebellum'),
    ('25905-2.0', 'Volume of grey matter in Crus II Cerebellum (right)'),
    ('25906-2.0', 'Volume of grey matter in VIIb Cerebellum (left)'),
    ('25907-2.0', 'Volume of grey matter in Vermis VIIb Cerebellum'),
    ('25908-2.0', 'Volume of grey matter in VIIb Cerebellum (right)'),
    ('25909-2.0', 'Volume of grey matter in VIIIa Cerebellum (left)'),
    ('25910-2.0', 'Volume of grey matter in Vermis VIIIa Cerebellum'),
    ('25911-2.0', 'Volume of grey matter in VIIIa Cerebellum (right)'),
    ('25912-2.0', 'Volume of grey matter in VIIIb Cerebellum (left)'),
    ('25913-2.0', 'Volume of grey matter in Vermis VIIIb Cerebellum'),
    ('25914-2.0', 'Volume of grey matter in VIIIb Cerebellum (right)'),
    ('25915-2.0', 'Volume of grey matter in IX Cerebellum (left)'),
    ('25916-2.0', 'Volume of grey matter in Vermis IX Cerebellum'),
    ('25917-2.0', 'Volume of grey matter in IX Cerebellum (right)'),
    ('25918-2.0', 'Volume of grey matter in X Cerebellum (left)'),
    ('25919-2.0', 'Volume of grey matter in Vermis X Cerebellum'),
    ('25920-2.0', 'Volume of grey matter in X Cerebellum (right)'),
])

brain_dmri_fa = OrderedDict([
    ('25079-2.0', 'Mean FA in anterior corona radiata on FA skeleton (left)'),
    ('25078-2.0', 'Mean FA in anterior corona radiata on FA skeleton (right)'),
    ('25073-2.0', 'Mean FA in anterior limb of internal capsule on FA skeleton (left)'),
    ('25072-2.0', 'Mean FA in anterior limb of internal capsule on FA skeleton (right)'),
    ('25059-2.0', 'Mean FA in body of corpus callosum on FA skeleton'),
    ('25071-2.0', 'Mean FA in cerebral peduncle on FA skeleton (left)'),
    ('25070-2.0', 'Mean FA in cerebral peduncle on FA skeleton (right)'),
    ('25091-2.0', 'Mean FA in cingulum cingulate gyrus on FA skeleton (left)'),
    ('25090-2.0', 'Mean FA in cingulum cingulate gyrus on FA skeleton (right)'),
    ('25093-2.0', 'Mean FA in cingulum hippocampus on FA skeleton (left)'),
    ('25092-2.0', 'Mean FA in cingulum hippocampus on FA skeleton (right)'),
    ('25063-2.0', 'Mean FA in corticospinal tract on FA skeleton (left)'),
    ('25062-2.0', 'Mean FA in corticospinal tract on FA skeleton (right)'),
    ('25089-2.0', 'Mean FA in external capsule on FA skeleton (left)'),
    ('25088-2.0', 'Mean FA in external capsule on FA skeleton (right)'),
    ('25095-2.0', 'Mean FA in fornix cres+stria terminalis on FA skeleton (left)'),
    ('25094-2.0', 'Mean FA in fornix cres+stria terminalis on FA skeleton (right)'),
    ('25061-2.0', 'Mean FA in fornix on FA skeleton'),
    ('25058-2.0', 'Mean FA in genu of corpus callosum on FA skeleton'),
    ('25067-2.0', 'Mean FA in inferior cerebellar peduncle on FA skeleton (left)'),
    ('25066-2.0', 'Mean FA in inferior cerebellar peduncle on FA skeleton (right)'),
    ('25065-2.0', 'Mean FA in medial lemniscus on FA skeleton (left)'),
    ('25064-2.0', 'Mean FA in medial lemniscus on FA skeleton (right)'),
    ('25056-2.0', 'Mean FA in middle cerebellar peduncle on FA skeleton'),
    ('25057-2.0', 'Mean FA in pontine crossing tract on FA skeleton'),
    ('25083-2.0', 'Mean FA in posterior corona radiata on FA skeleton (left)'),
    ('25082-2.0', 'Mean FA in posterior corona radiata on FA skeleton (right)'),
    ('25075-2.0', 'Mean FA in posterior limb of internal capsule on FA skeleton (left)'),
    ('25074-2.0', 'Mean FA in posterior limb of internal capsule on FA skeleton (right)'),
    ('25085-2.0', 'Mean FA in posterior thalamic radiation on FA skeleton (left)'),
    ('25084-2.0', 'Mean FA in posterior thalamic radiation on FA skeleton (right)'),
    ('25077-2.0', 'Mean FA in retrolenticular part of internal capsule on FA skeleton (left)'),
    ('25076-2.0', 'Mean FA in retrolenticular part of internal capsule on FA skeleton (right)'),
    ('25087-2.0', 'Mean FA in sagittal stratum on FA skeleton (left)'),
    ('25086-2.0', 'Mean FA in sagittal stratum on FA skeleton (right)'),
    ('25060-2.0', 'Mean FA in splenium of corpus callosum on FA skeleton'),
    ('25069-2.0', 'Mean FA in superior cerebellar peduncle on FA skeleton (left)'),
    ('25068-2.0', 'Mean FA in superior cerebellar peduncle on FA skeleton (right)'),
    ('25081-2.0', 'Mean FA in superior corona radiata on FA skeleton (left)'),
    ('25080-2.0', 'Mean FA in superior corona radiata on FA skeleton (right)'),
    ('25099-2.0', 'Mean FA in superior fronto-occipital fasciculus on FA skeleton (left)'),
    ('25098-2.0', 'Mean FA in superior fronto-occipital fasciculus on FA skeleton (right)'),
    ('25097-2.0', 'Mean FA in superior longitudinal fasciculus on FA skeleton (left)'),
    ('25096-2.0', 'Mean FA in superior longitudinal fasciculus on FA skeleton (right)'),
    ('25103-2.0', 'Mean FA in tapetum on FA skeleton (left)'),
    ('25102-2.0', 'Mean FA in tapetum on FA skeleton (right)'),
    ('25101-2.0', 'Mean FA in uncinate fasciculus on FA skeleton (left)'),
    ('25100-2.0', 'Mean FA in uncinate fasciculus on FA skeleton (right)'),
])

brain_dmri_icvf = OrderedDict([
    ('25367-2.0', 'Mean ICVF in anterior corona radiata on FA skeleton (left)'),
    ('25366-2.0', 'Mean ICVF in anterior corona radiata on FA skeleton (right)'),
    ('25361-2.0', 'Mean ICVF in anterior limb of internal capsule on FA skeleton (left)'),
    ('25360-2.0', 'Mean ICVF in anterior limb of internal capsule on FA skeleton (right)'),
    ('25347-2.0', 'Mean ICVF in body of corpus callosum on FA skeleton'),
    ('25359-2.0', 'Mean ICVF in cerebral peduncle on FA skeleton (left)'),
    ('25358-2.0', 'Mean ICVF in cerebral peduncle on FA skeleton (right)'),
    ('25379-2.0', 'Mean ICVF in cingulum cingulate gyrus on FA skeleton (left)'),
    ('25378-2.0', 'Mean ICVF in cingulum cingulate gyrus on FA skeleton (right)'),
    ('25381-2.0', 'Mean ICVF in cingulum hippocampus on FA skeleton (left)'),
    ('25380-2.0', 'Mean ICVF in cingulum hippocampus on FA skeleton (right)'),
    ('25351-2.0', 'Mean ICVF in corticospinal tract on FA skeleton (left)'),
    ('25350-2.0', 'Mean ICVF in corticospinal tract on FA skeleton (right)'),
    ('25377-2.0', 'Mean ICVF in external capsule on FA skeleton (left)'),
    ('25376-2.0', 'Mean ICVF in external capsule on FA skeleton (right)'),
    ('25383-2.0', 'Mean ICVF in fornix cres+stria terminalis on FA skeleton (left)'),
    ('25382-2.0', 'Mean ICVF in fornix cres+stria terminalis on FA skeleton (right)'),
    ('25349-2.0', 'Mean ICVF in fornix on FA skeleton'),
    ('25346-2.0', 'Mean ICVF in genu of corpus callosum on FA skeleton'),
    ('25355-2.0', 'Mean ICVF in inferior cerebellar peduncle on FA skeleton (left)'),
    ('25354-2.0', 'Mean ICVF in inferior cerebellar peduncle on FA skeleton (right)'),
    ('25353-2.0', 'Mean ICVF in medial lemniscus on FA skeleton (left)'),
    ('25352-2.0', 'Mean ICVF in medial lemniscus on FA skeleton (right)'),
    ('25344-2.0', 'Mean ICVF in middle cerebellar peduncle on FA skeleton'),
    ('25345-2.0', 'Mean ICVF in pontine crossing tract on FA skeleton'),
    ('25371-2.0', 'Mean ICVF in posterior corona radiata on FA skeleton (left)'),
    ('25370-2.0', 'Mean ICVF in posterior corona radiata on FA skeleton (right)'),
    ('25363-2.0', 'Mean ICVF in posterior limb of internal capsule on FA skeleton (left)'),
    ('25362-2.0', 'Mean ICVF in posterior limb of internal capsule on FA skeleton (right)'),
    ('25373-2.0', 'Mean ICVF in posterior thalamic radiation on FA skeleton (left)'),
    ('25372-2.0', 'Mean ICVF in posterior thalamic radiation on FA skeleton (right)'),
    ('25365-2.0', 'Mean ICVF in retrolenticular part of internal capsule on FA skeleton (left)'),
    ('25364-2.0', 'Mean ICVF in retrolenticular part of internal capsule on FA skeleton (right)'),
    ('25375-2.0', 'Mean ICVF in sagittal stratum on FA skeleton (left)'),
    ('25374-2.0', 'Mean ICVF in sagittal stratum on FA skeleton (right)'),
    ('25348-2.0', 'Mean ICVF in splenium of corpus callosum on FA skeleton'),
    ('25357-2.0', 'Mean ICVF in superior cerebellar peduncle on FA skeleton (left)'),
    ('25356-2.0', 'Mean ICVF in superior cerebellar peduncle on FA skeleton (right)'),
    ('25369-2.0', 'Mean ICVF in superior corona radiata on FA skeleton (left)'),
    ('25368-2.0', 'Mean ICVF in superior corona radiata on FA skeleton (right)'),
    ('25387-2.0', 'Mean ICVF in superior fronto-occipital fasciculus on FA skeleton (left)'),
    ('25386-2.0', 'Mean ICVF in superior fronto-occipital fasciculus on FA skeleton (right)'),
    ('25385-2.0', 'Mean ICVF in superior longitudinal fasciculus on FA skeleton (left)'),
    ('25384-2.0', 'Mean ICVF in superior longitudinal fasciculus on FA skeleton (right)'),
    ('25391-2.0', 'Mean ICVF in tapetum on FA skeleton (left)'),
    ('25390-2.0', 'Mean ICVF in tapetum on FA skeleton (right)'),
    ('25389-2.0', 'Mean ICVF in uncinate fasciculus on FA skeleton (left)'),
    ('25388-2.0', 'Mean ICVF in uncinate fasciculus on FA skeleton (right)'),
])

brain_dmri_isovf = OrderedDict([
    ('25463-2.0', 'Mean ISOVF in anterior corona radiata on FA skeleton (left)'),
    ('25462-2.0', 'Mean ISOVF in anterior corona radiata on FA skeleton (right)'),
    ('25457-2.0', 'Mean ISOVF in anterior limb of internal capsule on FA skeleton (left)'),
    ('25456-2.0', 'Mean ISOVF in anterior limb of internal capsule on FA skeleton (right)'),
    ('25443-2.0', 'Mean ISOVF in body of corpus callosum on FA skeleton'),
    ('25455-2.0', 'Mean ISOVF in cerebral peduncle on FA skeleton (left)'),
    ('25454-2.0', 'Mean ISOVF in cerebral peduncle on FA skeleton (right)'),
    ('25475-2.0', 'Mean ISOVF in cingulum cingulate gyrus on FA skeleton (left)'),
    ('25474-2.0', 'Mean ISOVF in cingulum cingulate gyrus on FA skeleton (right)'),
    ('25477-2.0', 'Mean ISOVF in cingulum hippocampus on FA skeleton (left)'),
    ('25476-2.0', 'Mean ISOVF in cingulum hippocampus on FA skeleton (right)'),
    ('25447-2.0', 'Mean ISOVF in corticospinal tract on FA skeleton (left)'),
    ('25446-2.0', 'Mean ISOVF in corticospinal tract on FA skeleton (right)'),
    ('25473-2.0', 'Mean ISOVF in external capsule on FA skeleton (left)'),
    ('25472-2.0', 'Mean ISOVF in external capsule on FA skeleton (right)'),
    ('25479-2.0', 'Mean ISOVF in fornix cres+stria terminalis on FA skeleton (left)'),
    ('25478-2.0', 'Mean ISOVF in fornix cres+stria terminalis on FA skeleton (right)'),
    ('25445-2.0', 'Mean ISOVF in fornix on FA skeleton'),
    ('25442-2.0', 'Mean ISOVF in genu of corpus callosum on FA skeleton'),
    ('25451-2.0', 'Mean ISOVF in inferior cerebellar peduncle on FA skeleton (left)'),
    ('25450-2.0', 'Mean ISOVF in inferior cerebellar peduncle on FA skeleton (right)'),
    ('25449-2.0', 'Mean ISOVF in medial lemniscus on FA skeleton (left)'),
    ('25448-2.0', 'Mean ISOVF in medial lemniscus on FA skeleton (right)'),
    ('25440-2.0', 'Mean ISOVF in middle cerebellar peduncle on FA skeleton'),
    ('25441-2.0', 'Mean ISOVF in pontine crossing tract on FA skeleton'),
    ('25467-2.0', 'Mean ISOVF in posterior corona radiata on FA skeleton (left)'),
    ('25466-2.0', 'Mean ISOVF in posterior corona radiata on FA skeleton (right)'),
    ('25459-2.0', 'Mean ISOVF in posterior limb of internal capsule on FA skeleton (left)'),
    ('25458-2.0', 'Mean ISOVF in posterior limb of internal capsule on FA skeleton (right)'),
    ('25469-2.0', 'Mean ISOVF in posterior thalamic radiation on FA skeleton (left)'),
    ('25468-2.0', 'Mean ISOVF in posterior thalamic radiation on FA skeleton (right)'),
    ('25461-2.0', 'Mean ISOVF in retrolenticular part of internal capsule on FA skeleton (left)'),
    ('25460-2.0', 'Mean ISOVF in retrolenticular part of internal capsule on FA skeleton (right)'),
    ('25471-2.0', 'Mean ISOVF in sagittal stratum on FA skeleton (left)'),
    ('25470-2.0', 'Mean ISOVF in sagittal stratum on FA skeleton (right)'),
    ('25444-2.0', 'Mean ISOVF in splenium of corpus callosum on FA skeleton'),
    ('25453-2.0', 'Mean ISOVF in superior cerebellar peduncle on FA skeleton (left)'),
    ('25452-2.0', 'Mean ISOVF in superior cerebellar peduncle on FA skeleton (right)'),
    ('25465-2.0', 'Mean ISOVF in superior corona radiata on FA skeleton (left)'),
    ('25464-2.0', 'Mean ISOVF in superior corona radiata on FA skeleton (right)'),
    ('25483-2.0', 'Mean ISOVF in superior fronto-occipital fasciculus on FA skeleton (left)'),
    ('25482-2.0', 'Mean ISOVF in superior fronto-occipital fasciculus on FA skeleton (right)'),
    ('25481-2.0', 'Mean ISOVF in superior longitudinal fasciculus on FA skeleton (left)'),
    ('25480-2.0', 'Mean ISOVF in superior longitudinal fasciculus on FA skeleton (right)'),
    ('25487-2.0', 'Mean ISOVF in tapetum on FA skeleton (left)'),
    ('25486-2.0', 'Mean ISOVF in tapetum on FA skeleton (right)'),
    ('25485-2.0', 'Mean ISOVF in uncinate fasciculus on FA skeleton (left)'),
    ('25484-2.0', 'Mean ISOVF in uncinate fasciculus on FA skeleton (right)'),
])

brain_dmri_l1 = OrderedDict([
    ('25223-2.0', 'Mean L1 in anterior corona radiata on FA skeleton (left)'),
    ('25222-2.0', 'Mean L1 in anterior corona radiata on FA skeleton (right)'),
    ('25217-2.0', 'Mean L1 in anterior limb of internal capsule on FA skeleton (left)'),
    ('25216-2.0', 'Mean L1 in anterior limb of internal capsule on FA skeleton (right)'),
    ('25203-2.0', 'Mean L1 in body of corpus callosum on FA skeleton'),
    ('25215-2.0', 'Mean L1 in cerebral peduncle on FA skeleton (left)'),
    ('25214-2.0', 'Mean L1 in cerebral peduncle on FA skeleton (right)'),
    ('25235-2.0', 'Mean L1 in cingulum cingulate gyrus on FA skeleton (left)'),
    ('25234-2.0', 'Mean L1 in cingulum cingulate gyrus on FA skeleton (right)'),
    ('25237-2.0', 'Mean L1 in cingulum hippocampus on FA skeleton (left)'),
    ('25236-2.0', 'Mean L1 in cingulum hippocampus on FA skeleton (right)'),
    ('25207-2.0', 'Mean L1 in corticospinal tract on FA skeleton (left)'),
    ('25206-2.0', 'Mean L1 in corticospinal tract on FA skeleton (right)'),
    ('25233-2.0', 'Mean L1 in external capsule on FA skeleton (left)'),
    ('25232-2.0', 'Mean L1 in external capsule on FA skeleton (right)'),
    ('25239-2.0', 'Mean L1 in fornix cres+stria terminalis on FA skeleton (left)'),
    ('25238-2.0', 'Mean L1 in fornix cres+stria terminalis on FA skeleton (right)'),
    ('25205-2.0', 'Mean L1 in fornix on FA skeleton'),
    ('25202-2.0', 'Mean L1 in genu of corpus callosum on FA skeleton'),
    ('25211-2.0', 'Mean L1 in inferior cerebellar peduncle on FA skeleton (left)'),
    ('25210-2.0', 'Mean L1 in inferior cerebellar peduncle on FA skeleton (right)'),
    ('25209-2.0', 'Mean L1 in medial lemniscus on FA skeleton (left)'),
    ('25208-2.0', 'Mean L1 in medial lemniscus on FA skeleton (right)'),
    ('25200-2.0', 'Mean L1 in middle cerebellar peduncle on FA skeleton'),
    ('25201-2.0', 'Mean L1 in pontine crossing tract on FA skeleton'),
    ('25227-2.0', 'Mean L1 in posterior corona radiata on FA skeleton (left)'),
    ('25226-2.0', 'Mean L1 in posterior corona radiata on FA skeleton (right)'),
    ('25219-2.0', 'Mean L1 in posterior limb of internal capsule on FA skeleton (left)'),
    ('25218-2.0', 'Mean L1 in posterior limb of internal capsule on FA skeleton (right)'),
    ('25229-2.0', 'Mean L1 in posterior thalamic radiation on FA skeleton (left)'),
    ('25228-2.0', 'Mean L1 in posterior thalamic radiation on FA skeleton (right)'),
    ('25221-2.0', 'Mean L1 in retrolenticular part of internal capsule on FA skeleton (left)'),
    ('25220-2.0', 'Mean L1 in retrolenticular part of internal capsule on FA skeleton (right)'),
    ('25231-2.0', 'Mean L1 in sagittal stratum on FA skeleton (left)'),
    ('25230-2.0', 'Mean L1 in sagittal stratum on FA skeleton (right)'),
    ('25204-2.0', 'Mean L1 in splenium of corpus callosum on FA skeleton'),
    ('25213-2.0', 'Mean L1 in superior cerebellar peduncle on FA skeleton (left)'),
    ('25212-2.0', 'Mean L1 in superior cerebellar peduncle on FA skeleton (right)'),
    ('25225-2.0', 'Mean L1 in superior corona radiata on FA skeleton (left)'),
    ('25224-2.0', 'Mean L1 in superior corona radiata on FA skeleton (right)'),
    ('25243-2.0', 'Mean L1 in superior fronto-occipital fasciculus on FA skeleton (left)'),
    ('25242-2.0', 'Mean L1 in superior fronto-occipital fasciculus on FA skeleton (right)'),
    ('25241-2.0', 'Mean L1 in superior longitudinal fasciculus on FA skeleton (left)'),
    ('25240-2.0', 'Mean L1 in superior longitudinal fasciculus on FA skeleton (right)'),
    ('25247-2.0', 'Mean L1 in tapetum on FA skeleton (left)'),
    ('25246-2.0', 'Mean L1 in tapetum on FA skeleton (right)'),
    ('25245-2.0', 'Mean L1 in uncinate fasciculus on FA skeleton (left)'),
    ('25244-2.0', 'Mean L1 in uncinate fasciculus on FA skeleton (right)'),
])

brain_dmri_l2 = OrderedDict([
    ('25271-2.0', 'Mean L2 in anterior corona radiata on FA skeleton (left)'),
    ('25270-2.0', 'Mean L2 in anterior corona radiata on FA skeleton (right)'),
    ('25265-2.0', 'Mean L2 in anterior limb of internal capsule on FA skeleton (left)'),
    ('25264-2.0', 'Mean L2 in anterior limb of internal capsule on FA skeleton (right)'),
    ('25251-2.0', 'Mean L2 in body of corpus callosum on FA skeleton'),
    ('25263-2.0', 'Mean L2 in cerebral peduncle on FA skeleton (left)'),
    ('25262-2.0', 'Mean L2 in cerebral peduncle on FA skeleton (right)'),
    ('25283-2.0', 'Mean L2 in cingulum cingulate gyrus on FA skeleton (left)'),
    ('25282-2.0', 'Mean L2 in cingulum cingulate gyrus on FA skeleton (right)'),
    ('25285-2.0', 'Mean L2 in cingulum hippocampus on FA skeleton (left)'),
    ('25284-2.0', 'Mean L2 in cingulum hippocampus on FA skeleton (right)'),
    ('25255-2.0', 'Mean L2 in corticospinal tract on FA skeleton (left)'),
    ('25254-2.0', 'Mean L2 in corticospinal tract on FA skeleton (right)'),
    ('25281-2.0', 'Mean L2 in external capsule on FA skeleton (left)'),
    ('25280-2.0', 'Mean L2 in external capsule on FA skeleton (right)'),
    ('25287-2.0', 'Mean L2 in fornix cres+stria terminalis on FA skeleton (left)'),
    ('25286-2.0', 'Mean L2 in fornix cres+stria terminalis on FA skeleton (right)'),
    ('25253-2.0', 'Mean L2 in fornix on FA skeleton'),
    ('25250-2.0', 'Mean L2 in genu of corpus callosum on FA skeleton'),
    ('25259-2.0', 'Mean L2 in inferior cerebellar peduncle on FA skeleton (left)'),
    ('25258-2.0', 'Mean L2 in inferior cerebellar peduncle on FA skeleton (right)'),
    ('25257-2.0', 'Mean L2 in medial lemniscus on FA skeleton (left)'),
    ('25256-2.0', 'Mean L2 in medial lemniscus on FA skeleton (right)'),
    ('25248-2.0', 'Mean L2 in middle cerebellar peduncle on FA skeleton'),
    ('25249-2.0', 'Mean L2 in pontine crossing tract on FA skeleton'),
    ('25275-2.0', 'Mean L2 in posterior corona radiata on FA skeleton (left)'),
    ('25274-2.0', 'Mean L2 in posterior corona radiata on FA skeleton (right)'),
    ('25267-2.0', 'Mean L2 in posterior limb of internal capsule on FA skeleton (left)'),
    ('25266-2.0', 'Mean L2 in posterior limb of internal capsule on FA skeleton (right)'),
    ('25277-2.0', 'Mean L2 in posterior thalamic radiation on FA skeleton (left)'),
    ('25276-2.0', 'Mean L2 in posterior thalamic radiation on FA skeleton (right)'),
    ('25269-2.0', 'Mean L2 in retrolenticular part of internal capsule on FA skeleton (left)'),
    ('25268-2.0', 'Mean L2 in retrolenticular part of internal capsule on FA skeleton (right)'),
    ('25279-2.0', 'Mean L2 in sagittal stratum on FA skeleton (left)'),
    ('25278-2.0', 'Mean L2 in sagittal stratum on FA skeleton (right)'),
    ('25252-2.0', 'Mean L2 in splenium of corpus callosum on FA skeleton'),
    ('25261-2.0', 'Mean L2 in superior cerebellar peduncle on FA skeleton (left)'),
    ('25260-2.0', 'Mean L2 in superior cerebellar peduncle on FA skeleton (right)'),
    ('25273-2.0', 'Mean L2 in superior corona radiata on FA skeleton (left)'),
    ('25272-2.0', 'Mean L2 in superior corona radiata on FA skeleton (right)'),
    ('25291-2.0', 'Mean L2 in superior fronto-occipital fasciculus on FA skeleton (left)'),
    ('25290-2.0', 'Mean L2 in superior fronto-occipital fasciculus on FA skeleton (right)'),
    ('25289-2.0', 'Mean L2 in superior longitudinal fasciculus on FA skeleton (left)'),
    ('25288-2.0', 'Mean L2 in superior longitudinal fasciculus on FA skeleton (right)'),
    ('25295-2.0', 'Mean L2 in tapetum on FA skeleton (left)'),
    ('25294-2.0', 'Mean L2 in tapetum on FA skeleton (right)'),
    ('25293-2.0', 'Mean L2 in uncinate fasciculus on FA skeleton (left)'),
    ('25292-2.0', 'Mean L2 in uncinate fasciculus on FA skeleton (right)'),
])

brain_dmri_l3 = OrderedDict([
    ('25319-2.0', 'Mean L3 in anterior corona radiata on FA skeleton (left)'),
    ('25318-2.0', 'Mean L3 in anterior corona radiata on FA skeleton (right)'),
    ('25313-2.0', 'Mean L3 in anterior limb of internal capsule on FA skeleton (left)'),
    ('25312-2.0', 'Mean L3 in anterior limb of internal capsule on FA skeleton (right)'),
    ('25299-2.0', 'Mean L3 in body of corpus callosum on FA skeleton'),
    ('25311-2.0', 'Mean L3 in cerebral peduncle on FA skeleton (left)'),
    ('25310-2.0', 'Mean L3 in cerebral peduncle on FA skeleton (right)'),
    ('25331-2.0', 'Mean L3 in cingulum cingulate gyrus on FA skeleton (left)'),
    ('25330-2.0', 'Mean L3 in cingulum cingulate gyrus on FA skeleton (right)'),
    ('25333-2.0', 'Mean L3 in cingulum hippocampus on FA skeleton (left)'),
    ('25332-2.0', 'Mean L3 in cingulum hippocampus on FA skeleton (right)'),
    ('25303-2.0', 'Mean L3 in corticospinal tract on FA skeleton (left)'),
    ('25302-2.0', 'Mean L3 in corticospinal tract on FA skeleton (right)'),
    ('25329-2.0', 'Mean L3 in external capsule on FA skeleton (left)'),
    ('25328-2.0', 'Mean L3 in external capsule on FA skeleton (right)'),
    ('25335-2.0', 'Mean L3 in fornix cres+stria terminalis on FA skeleton (left)'),
    ('25334-2.0', 'Mean L3 in fornix cres+stria terminalis on FA skeleton (right)'),
    ('25301-2.0', 'Mean L3 in fornix on FA skeleton'),
    ('25298-2.0', 'Mean L3 in genu of corpus callosum on FA skeleton'),
    ('25307-2.0', 'Mean L3 in inferior cerebellar peduncle on FA skeleton (left)'),
    ('25306-2.0', 'Mean L3 in inferior cerebellar peduncle on FA skeleton (right)'),
    ('25305-2.0', 'Mean L3 in medial lemniscus on FA skeleton (left)'),
    ('25304-2.0', 'Mean L3 in medial lemniscus on FA skeleton (right)'),
    ('25296-2.0', 'Mean L3 in middle cerebellar peduncle on FA skeleton'),
    ('25297-2.0', 'Mean L3 in pontine crossing tract on FA skeleton'),
    ('25323-2.0', 'Mean L3 in posterior corona radiata on FA skeleton (left)'),
    ('25322-2.0', 'Mean L3 in posterior corona radiata on FA skeleton (right)'),
    ('25315-2.0', 'Mean L3 in posterior limb of internal capsule on FA skeleton (left)'),
    ('25314-2.0', 'Mean L3 in posterior limb of internal capsule on FA skeleton (right)'),
    ('25325-2.0', 'Mean L3 in posterior thalamic radiation on FA skeleton (left)'),
    ('25324-2.0', 'Mean L3 in posterior thalamic radiation on FA skeleton (right)'),
    ('25317-2.0', 'Mean L3 in retrolenticular part of internal capsule on FA skeleton (left)'),
    ('25316-2.0', 'Mean L3 in retrolenticular part of internal capsule on FA skeleton (right)'),
    ('25327-2.0', 'Mean L3 in sagittal stratum on FA skeleton (left)'),
    ('25326-2.0', 'Mean L3 in sagittal stratum on FA skeleton (right)'),
    ('25300-2.0', 'Mean L3 in splenium of corpus callosum on FA skeleton'),
    ('25309-2.0', 'Mean L3 in superior cerebellar peduncle on FA skeleton (left)'),
    ('25308-2.0', 'Mean L3 in superior cerebellar peduncle on FA skeleton (right)'),
    ('25321-2.0', 'Mean L3 in superior corona radiata on FA skeleton (left)'),
    ('25320-2.0', 'Mean L3 in superior corona radiata on FA skeleton (right)'),
    ('25339-2.0', 'Mean L3 in superior fronto-occipital fasciculus on FA skeleton (left)'),
    ('25338-2.0', 'Mean L3 in superior fronto-occipital fasciculus on FA skeleton (right)'),
    ('25337-2.0', 'Mean L3 in superior longitudinal fasciculus on FA skeleton (left)'),
    ('25336-2.0', 'Mean L3 in superior longitudinal fasciculus on FA skeleton (right)'),
    ('25343-2.0', 'Mean L3 in tapetum on FA skeleton (left)'),
    ('25342-2.0', 'Mean L3 in tapetum on FA skeleton (right)'),
    ('25341-2.0', 'Mean L3 in uncinate fasciculus on FA skeleton (left)'),
    ('25340-2.0', 'Mean L3 in uncinate fasciculus on FA skeleton (right)'),
])

brain_dmri_md = OrderedDict([
    ('25127-2.0', 'Mean MD in anterior corona radiata on FA skeleton (left)'),
    ('25126-2.0', 'Mean MD in anterior corona radiata on FA skeleton (right)'),
    ('25121-2.0', 'Mean MD in anterior limb of internal capsule on FA skeleton (left)'),
    ('25120-2.0', 'Mean MD in anterior limb of internal capsule on FA skeleton (right)'),
    ('25107-2.0', 'Mean MD in body of corpus callosum on FA skeleton'),
    ('25119-2.0', 'Mean MD in cerebral peduncle on FA skeleton (left)'),
    ('25118-2.0', 'Mean MD in cerebral peduncle on FA skeleton (right)'),
    ('25139-2.0', 'Mean MD in cingulum cingulate gyrus on FA skeleton (left)'),
    ('25138-2.0', 'Mean MD in cingulum cingulate gyrus on FA skeleton (right)'),
    ('25141-2.0', 'Mean MD in cingulum hippocampus on FA skeleton (left)'),
    ('25140-2.0', 'Mean MD in cingulum hippocampus on FA skeleton (right)'),
    ('25111-2.0', 'Mean MD in corticospinal tract on FA skeleton (left)'),
    ('25110-2.0', 'Mean MD in corticospinal tract on FA skeleton (right)'),
    ('25137-2.0', 'Mean MD in external capsule on FA skeleton (left)'),
    ('25136-2.0', 'Mean MD in external capsule on FA skeleton (right)'),
    ('25143-2.0', 'Mean MD in fornix cres+stria terminalis on FA skeleton (left)'),
    ('25142-2.0', 'Mean MD in fornix cres+stria terminalis on FA skeleton (right)'),
    ('25109-2.0', 'Mean MD in fornix on FA skeleton'),
    ('25106-2.0', 'Mean MD in genu of corpus callosum on FA skeleton'),
    ('25115-2.0', 'Mean MD in inferior cerebellar peduncle on FA skeleton (left)'),
    ('25114-2.0', 'Mean MD in inferior cerebellar peduncle on FA skeleton (right)'),
    ('25113-2.0', 'Mean MD in medial lemniscus on FA skeleton (left)'),
    ('25112-2.0', 'Mean MD in medial lemniscus on FA skeleton (right)'),
    ('25104-2.0', 'Mean MD in middle cerebellar peduncle on FA skeleton'),
    ('25105-2.0', 'Mean MD in pontine crossing tract on FA skeleton'),
    ('25131-2.0', 'Mean MD in posterior corona radiata on FA skeleton (left)'),
    ('25130-2.0', 'Mean MD in posterior corona radiata on FA skeleton (right)'),
    ('25123-2.0', 'Mean MD in posterior limb of internal capsule on FA skeleton (left)'),
    ('25122-2.0', 'Mean MD in posterior limb of internal capsule on FA skeleton (right)'),
    ('25133-2.0', 'Mean MD in posterior thalamic radiation on FA skeleton (left)'),
    ('25132-2.0', 'Mean MD in posterior thalamic radiation on FA skeleton (right)'),
    ('25125-2.0', 'Mean MD in retrolenticular part of internal capsule on FA skeleton (left)'),
    ('25124-2.0', 'Mean MD in retrolenticular part of internal capsule on FA skeleton (right)'),
    ('25135-2.0', 'Mean MD in sagittal stratum on FA skeleton (left)'),
    ('25134-2.0', 'Mean MD in sagittal stratum on FA skeleton (right)'),
    ('25108-2.0', 'Mean MD in splenium of corpus callosum on FA skeleton'),
    ('25117-2.0', 'Mean MD in superior cerebellar peduncle on FA skeleton (left)'),
    ('25116-2.0', 'Mean MD in superior cerebellar peduncle on FA skeleton (right)'),
    ('25129-2.0', 'Mean MD in superior corona radiata on FA skeleton (left)'),
    ('25128-2.0', 'Mean MD in superior corona radiata on FA skeleton (right)'),
    ('25147-2.0', 'Mean MD in superior fronto-occipital fasciculus on FA skeleton (left)'),
    ('25146-2.0', 'Mean MD in superior fronto-occipital fasciculus on FA skeleton (right)'),
    ('25145-2.0', 'Mean MD in superior longitudinal fasciculus on FA skeleton (left)'),
    ('25144-2.0', 'Mean MD in superior longitudinal fasciculus on FA skeleton (right)'),
    ('25151-2.0', 'Mean MD in tapetum on FA skeleton (left)'),
    ('25150-2.0', 'Mean MD in tapetum on FA skeleton (right)'),
    ('25149-2.0', 'Mean MD in uncinate fasciculus on FA skeleton (left)'),
    ('25148-2.0', 'Mean MD in uncinate fasciculus on FA skeleton (right)'),
])

brain_dmri_mo = OrderedDict([
    ('25175-2.0', 'Mean MO in anterior corona radiata on FA skeleton (left)'),
    ('25174-2.0', 'Mean MO in anterior corona radiata on FA skeleton (right)'),
    ('25169-2.0', 'Mean MO in anterior limb of internal capsule on FA skeleton (left)'),
    ('25168-2.0', 'Mean MO in anterior limb of internal capsule on FA skeleton (right)'),
    ('25155-2.0', 'Mean MO in body of corpus callosum on FA skeleton'),
    ('25167-2.0', 'Mean MO in cerebral peduncle on FA skeleton (left)'),
    ('25166-2.0', 'Mean MO in cerebral peduncle on FA skeleton (right)'),
    ('25187-2.0', 'Mean MO in cingulum cingulate gyrus on FA skeleton (left)'),
    ('25186-2.0', 'Mean MO in cingulum cingulate gyrus on FA skeleton (right)'),
    ('25189-2.0', 'Mean MO in cingulum hippocampus on FA skeleton (left)'),
    ('25188-2.0', 'Mean MO in cingulum hippocampus on FA skeleton (right)'),
    ('25159-2.0', 'Mean MO in corticospinal tract on FA skeleton (left)'),
    ('25158-2.0', 'Mean MO in corticospinal tract on FA skeleton (right)'),
    ('25185-2.0', 'Mean MO in external capsule on FA skeleton (left)'),
    ('25184-2.0', 'Mean MO in external capsule on FA skeleton (right)'),
    ('25191-2.0', 'Mean MO in fornix cres+stria terminalis on FA skeleton (left)'),
    ('25190-2.0', 'Mean MO in fornix cres+stria terminalis on FA skeleton (right)'),
    ('25157-2.0', 'Mean MO in fornix on FA skeleton'),
    ('25154-2.0', 'Mean MO in genu of corpus callosum on FA skeleton'),
    ('25163-2.0', 'Mean MO in inferior cerebellar peduncle on FA skeleton (left)'),
    ('25162-2.0', 'Mean MO in inferior cerebellar peduncle on FA skeleton (right)'),
    ('25161-2.0', 'Mean MO in medial lemniscus on FA skeleton (left)'),
    ('25160-2.0', 'Mean MO in medial lemniscus on FA skeleton (right)'),
    ('25152-2.0', 'Mean MO in middle cerebellar peduncle on FA skeleton'),
    ('25153-2.0', 'Mean MO in pontine crossing tract on FA skeleton'),
    ('25179-2.0', 'Mean MO in posterior corona radiata on FA skeleton (left)'),
    ('25178-2.0', 'Mean MO in posterior corona radiata on FA skeleton (right)'),
    ('25171-2.0', 'Mean MO in posterior limb of internal capsule on FA skeleton (left)'),
    ('25170-2.0', 'Mean MO in posterior limb of internal capsule on FA skeleton (right)'),
    ('25181-2.0', 'Mean MO in posterior thalamic radiation on FA skeleton (left)'),
    ('25180-2.0', 'Mean MO in posterior thalamic radiation on FA skeleton (right)'),
    ('25173-2.0', 'Mean MO in retrolenticular part of internal capsule on FA skeleton (left)'),
    ('25172-2.0', 'Mean MO in retrolenticular part of internal capsule on FA skeleton (right)'),
    ('25183-2.0', 'Mean MO in sagittal stratum on FA skeleton (left)'),
    ('25182-2.0', 'Mean MO in sagittal stratum on FA skeleton (right)'),
    ('25156-2.0', 'Mean MO in splenium of corpus callosum on FA skeleton'),
    ('25165-2.0', 'Mean MO in superior cerebellar peduncle on FA skeleton (left)'),
    ('25164-2.0', 'Mean MO in superior cerebellar peduncle on FA skeleton (right)'),
    ('25177-2.0', 'Mean MO in superior corona radiata on FA skeleton (left)'),
    ('25176-2.0', 'Mean MO in superior corona radiata on FA skeleton (right)'),
    ('25195-2.0', 'Mean MO in superior fronto-occipital fasciculus on FA skeleton (left)'),
    ('25194-2.0', 'Mean MO in superior fronto-occipital fasciculus on FA skeleton (right)'),
    ('25193-2.0', 'Mean MO in superior longitudinal fasciculus on FA skeleton (left)'),
    ('25192-2.0', 'Mean MO in superior longitudinal fasciculus on FA skeleton (right)'),
    ('25199-2.0', 'Mean MO in tapetum on FA skeleton (left)'),
    ('25198-2.0', 'Mean MO in tapetum on FA skeleton (right)'),
    ('25197-2.0', 'Mean MO in uncinate fasciculus on FA skeleton (left)'),
    ('25196-2.0', 'Mean MO in uncinate fasciculus on FA skeleton (right)'),
])

brain_dmri_od = OrderedDict([
    ('25415-2.0', 'Mean OD in anterior corona radiata on FA skeleton (left)'),
    ('25414-2.0', 'Mean OD in anterior corona radiata on FA skeleton (right)'),
    ('25409-2.0', 'Mean OD in anterior limb of internal capsule on FA skeleton (left)'),
    ('25408-2.0', 'Mean OD in anterior limb of internal capsule on FA skeleton (right)'),
    ('25395-2.0', 'Mean OD in body of corpus callosum on FA skeleton'),
    ('25407-2.0', 'Mean OD in cerebral peduncle on FA skeleton (left)'),
    ('25406-2.0', 'Mean OD in cerebral peduncle on FA skeleton (right)'),
    ('25427-2.0', 'Mean OD in cingulum cingulate gyrus on FA skeleton (left)'),
    ('25426-2.0', 'Mean OD in cingulum cingulate gyrus on FA skeleton (right)'),
    ('25429-2.0', 'Mean OD in cingulum hippocampus on FA skeleton (left)'),
    ('25428-2.0', 'Mean OD in cingulum hippocampus on FA skeleton (right)'),
    ('25399-2.0', 'Mean OD in corticospinal tract on FA skeleton (left)'),
    ('25398-2.0', 'Mean OD in corticospinal tract on FA skeleton (right)'),
    ('25425-2.0', 'Mean OD in external capsule on FA skeleton (left)'),
    ('25424-2.0', 'Mean OD in external capsule on FA skeleton (right)'),
    ('25431-2.0', 'Mean OD in fornix cres+stria terminalis on FA skeleton (left)'),
    ('25430-2.0', 'Mean OD in fornix cres+stria terminalis on FA skeleton (right)'),
    ('25397-2.0', 'Mean OD in fornix on FA skeleton'),
    ('25394-2.0', 'Mean OD in genu of corpus callosum on FA skeleton'),
    ('25403-2.0', 'Mean OD in inferior cerebellar peduncle on FA skeleton (left)'),
    ('25402-2.0', 'Mean OD in inferior cerebellar peduncle on FA skeleton (right)'),
    ('25401-2.0', 'Mean OD in medial lemniscus on FA skeleton (left)'),
    ('25400-2.0', 'Mean OD in medial lemniscus on FA skeleton (right)'),
    ('25392-2.0', 'Mean OD in middle cerebellar peduncle on FA skeleton'),
    ('25393-2.0', 'Mean OD in pontine crossing tract on FA skeleton'),
    ('25419-2.0', 'Mean OD in posterior corona radiata on FA skeleton (left)'),
    ('25418-2.0', 'Mean OD in posterior corona radiata on FA skeleton (right)'),
    ('25411-2.0', 'Mean OD in posterior limb of internal capsule on FA skeleton (left)'),
    ('25410-2.0', 'Mean OD in posterior limb of internal capsule on FA skeleton (right)'),
    ('25421-2.0', 'Mean OD in posterior thalamic radiation on FA skeleton (left)'),
    ('25420-2.0', 'Mean OD in posterior thalamic radiation on FA skeleton (right)'),
    ('25413-2.0', 'Mean OD in retrolenticular part of internal capsule on FA skeleton (left)'),
    ('25412-2.0', 'Mean OD in retrolenticular part of internal capsule on FA skeleton (right)'),
    ('25423-2.0', 'Mean OD in sagittal stratum on FA skeleton (left)'),
    ('25422-2.0', 'Mean OD in sagittal stratum on FA skeleton (right)'),
    ('25396-2.0', 'Mean OD in splenium of corpus callosum on FA skeleton'),
    ('25405-2.0', 'Mean OD in superior cerebellar peduncle on FA skeleton (left)'),
    ('25404-2.0', 'Mean OD in superior cerebellar peduncle on FA skeleton (right)'),
    ('25417-2.0', 'Mean OD in superior corona radiata on FA skeleton (left)'),
    ('25416-2.0', 'Mean OD in superior corona radiata on FA skeleton (right)'),
    ('25435-2.0', 'Mean OD in superior fronto-occipital fasciculus on FA skeleton (left)'),
    ('25434-2.0', 'Mean OD in superior fronto-occipital fasciculus on FA skeleton (right)'),
    ('25433-2.0', 'Mean OD in superior longitudinal fasciculus on FA skeleton (left)'),
    ('25432-2.0', 'Mean OD in superior longitudinal fasciculus on FA skeleton (right)'),
    ('25439-2.0', 'Mean OD in tapetum on FA skeleton (left)'),
    ('25438-2.0', 'Mean OD in tapetum on FA skeleton (right)'),
    ('25437-2.0', 'Mean OD in uncinate fasciculus on FA skeleton (left)'),
    ('25436-2.0', 'Mean OD in uncinate fasciculus on FA skeleton (right))'),
])


In [3]:
codes = OrderedDict(list(brain_dmri_fa.items()) + list(brain_dmri_icvf.items()) + \
                    list(brain_dmri_isovf.items()) + list(brain_dmri_l1.items()) + \
                    list(brain_dmri_l2.items()) + list(brain_dmri_l3.items()) + \
                    list(brain_dmri_md.items()) + list(brain_dmri_mo.items()) + \
                    list(brain_dmri_od.items()) + list(brain_smri_plus.items()) + \
                    list(earlylife.items()) + list(fluid_intelligence.items()) + \
                    list(brain_smri.items()) + list(education.items()) + \
                    list(lifestyle.items()) + list(mental_health.items()))



In [4]:
codes = (brain_dmri_fa | brain_dmri_icvf | brain_dmri_isovf | brain_dmri_l1 | brain_dmri_l2 |
         brain_dmri_l3 | brain_dmri_md | brain_dmri_mo | brain_dmri_od | brain_smri_plus | earlylife |
         fluid_intelligence | brain_smri | education | lifestyle | mental_health)

In [5]:
in_path = pathlib.Path('../ukbb/Results_variables')
list_outcomes = ['Age', 'Alcohol', 'Gfactor', 'Intelligence', 'Met', 'Neuro', 'Sleep', 'Smoke']
brain_variables = ['25056-2.0', '25057-2.0', '25060-2.0', '25059-2.0', '25061-2.0', '25058-2.0', '25009-2.0', '25025-2.0',
                   '25898-2.0', '25901-2.0', '25904-2.0', '25907-2.0', '25910-2.0', '25913-2.0', '25916-2.0', '25919-2.0',
                   '25396-2.0', '25393-2.0', '25392-2.0', '25394-2.0', '25397-2.0', '25395-2.0', '25156-2.0', '25153-2.0',
                   '25152-2.0', '25154-2.0', '25157-2.0', '25108-2.0', '25105-2.0', '25104-2.0', '25106-2.0', '25109-2.0',
                   '25107-2.0', '25300-2.0', '25297-2.0', '25296-2.0', '25298-2.0', '25301-2.0', '25299-2.0', '25252-2.0',
                   '25251-2.0', '25253-2.0', '25250-2.0', '25248-2.0', '25249-2.0', '25203-2.0', '25205-2.0', '25202-2.0',
                   '25200-2.0', '25201-2.0', '25204-2.0', '25443-2.0', '25445-2.0', '25442-2.0', '25440-2.0', '25441-2.0',
                   '25444-2.0', '25347-2.0', '25349-2.0', '25346-2.0', '25344-2.0', '25345-2.0', '25348-2.0', '25892-2.0',
                   '25001-2.0', '25005-2.0']

socio_variables = ['31-0.0', '21022-0.0', '20016-2.0', '20127-0.0', '1677-2.0', '1787-2.0',
                   '6138-2.0', '709-2.0', '738-2.0', '2040-2.0', '4526-2.0', '4537-2.0', '4548-2.0', '6142-2.0',
                   '4559-2.0', '4570-2.0', '4581-2.0', '670-2.0', '699-2.0', '767-2.0', '806-2.0', '816-2.0',
                   '3426-2.0', '6160-2.0', '4631-2.0', '6156-2.0', '6145-2.0', '4598-2.0', '4609-2.0', '4620-2.0']

data_list = list()
for outcome in list_outcomes:
    df_permfit = pd.read_csv (in_path / f"Result_{outcome}_permfit_single.csv")[['variable', 'p_value', 'score']]
    df_cpi = pd.read_csv(in_path / f"Result_{outcome}_cpi_single.csv")[['variable', 'p_value', 'score']]
    df = df_permfit.merge(df_cpi, on='variable', suffixes=('_permfit', '_cpi'))
    df['cpi_rank'] = df['p_value_cpi'].rank()
    df['permfit_rank'] = df['p_value_permfit'].rank()
    df['outcome'] = outcome
    data_list.append(df)
df_main = pd.concat(data_list)
df_main['variable_name'] = df_main['variable'].map(codes)
df_main['domain'] = df_main.variable.isin(brain_variables).map({True: 'Brain', False: 'Social'})

In [6]:
df_main.query("outcome == 'age'")
df_main['scaled_diff'] = -np.sqrt(np.abs(df_main['p_value_permfit']-df_main['p_value_cpi']))
df_main['mlog10_p_value_permfit'] = -np.log10(df_main['p_value_permfit'])
df_main['mlog10_p_value_cpi'] = -np.log10(df_main['p_value_cpi'])
df_main['mlog10_p_value_cpi'] = -np.log10(df_main['p_value_cpi'])

## Prepare figures

In [7]:
df_main['Relevance'] = 'Unimportant'
df_main['Relevance'].iloc[
    ((df_main['p_value_cpi'] < 0.05) &
     (df_main['p_value_permfit'] < 0.05))] = 'Important'
df_main['Relevance'].iloc[
    ((df_main['p_value_cpi'] < 0.05) &
     (df_main['p_value_permfit'] >= 0.05))] = 'Cpi'
df_main['Relevance'].iloc[
    ((df_main['p_value_cpi'] >= 0.05) &
     (df_main['p_value_permfit'] < 0.05))] = 'Permfit'
df_main['Relevance'].unique()

/tmp/ipykernel_21058/3010316638.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_main['Relevance'].iloc[
/tmp/ipykernel_21058/3010316638.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_main['Relevance'].iloc[
/tmp/ipykernel_21058/3010316638.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_main['Relevance'].iloc[


array(['Important', 'Permfit', 'Unimportant', 'Cpi'], dtype=object)

In [8]:
 df_main.query("outcome == 'Age'")[['p_value_permfit', 'p_value_cpi']].corr()

,p_value_permfit,p_value_cpi
p_value_permfit,1.000000,0.633788
p_value_cpi,0.633788,1.000000


In [9]:
df_main['rank_diff'] = abs(df_main['permfit_rank'] - df_main['cpi_rank'])

In [10]:
change_names = {"Age": "Age", "Alcohol": "Alcohol", "Intelligence": "Cognition", "Neuro": "Mood",
                "Met": "Exercise", "Smoke": "Smoke", "Sleep": "Sleep"}
df_main['outcome'] = df_main['outcome'].map(change_names)

In [11]:
df_count = df_main.groupby(['outcome', 'Relevance', 'domain'])['rank_diff'].count().reset_index()
df_count.columns = ['outcome', 'Relevance', 'domain', 'count']
df_count

,outcome,Relevance,domain,count
0,Age,Cpi,Brain,3
1,Age,Important,Brain,23
2,Age,Important,Social,17
3,Age,Permfit,Brain,16
4,Age,Permfit,Social,3
5,Age,Unimportant,Brain,24
6,Age,Unimportant,Social,8
7,Alcohol,Cpi,Brain,2
8,Alcohol,Cpi,Social,3
9,Alcohol,Important,Brain,2


In [12]:
df_count_brain = df_count[df_count['domain'] == 'Brain']
df_count_social = df_count[df_count['domain'] == 'Social']
df_count_brain

,outcome,Relevance,domain,count
0,Age,Cpi,Brain,3
1,Age,Important,Brain,23
3,Age,Permfit,Brain,16
5,Age,Unimportant,Brain,24
7,Alcohol,Cpi,Brain,2
9,Alcohol,Important,Brain,2
11,Alcohol,Permfit,Brain,5
13,Alcohol,Unimportant,Brain,57
15,Cognition,Cpi,Brain,5
17,Cognition,Important,Brain,5


In [13]:
scheme = 'dark2'
scatter = alt.Chart(
    df_main.query('outcome!="Gfactor"')
).mark_point(
    opacity=0.3
).encode(
    x=alt.X('permfit_rank:Q',
            title='Rank (Permfit-DNN)',
            axis=alt.Axis(tickCount=10,
                         labelExpr="datum.value % 10 ? null : datum.label"),
            scale=alt.Scale(type='sqrt', nice=False)),
    y=alt.Y('cpi_rank:Q',
            title='Rank (CPI-DNN)',
            axis=alt.Axis(tickCount=10,
                          labelExpr="datum.value % 10 ? null : datum.label"),
            scale=alt.Scale(type='sqrt', nice=False)),
    color=alt.Color('domain', scale=alt.Scale(scheme=scheme)),
    shape='Relevance',
    opacity=alt.Opacity('cpi_rank', legend=None, scale=alt.Scale(domain=[100, 0])),
    tooltip=['variable_name'],
).interactive(
).properties(
    height=300,
    width=300
).properties(title=alt.TitleParams('A', anchor='start'))

def plot_bar(data, title='', title_y=None, labels_y=False):
    scheme = 'category20'
    bar = alt.Chart(
        data.query('outcome != "Gfactor"')
    ).mark_bar(
        size=25
    ).encode(
        x=alt.X('count:Q', stack='normalize', title='Percentage',
                axis=alt.Axis(grid=False, format='%')),
        y=alt.Y('outcome', title=title_y, axis=alt.Axis(labels=labels_y)),
        color=alt.Color('Relevance:N', scale=alt.Scale(scheme=scheme)),
        order='Relevance:N'
    ).properties(
        width=350,
        height=310
    ).properties(title=alt.TitleParams(title, anchor='start'))
    text = bar.mark_text(
        dx=-8, dy=2, size=15
    ).encode(
        x=alt.X('count:Q', stack='normalize', title='Percentage',
                axis=alt.Axis(grid=False)),
        text=alt.Text('count:Q', format='.1d'),
        color=alt.value('white'),
        detail='Relevance'
    )

    bars = (bar + text).facet(
       column=alt.Column('domain', header=alt.Header(title=None))
    )
    return bars

bar1 = plot_bar(df_count_brain, 'B', 'Outcome', True)
bar2 = plot_bar(df_count_social, ' ', None, False)
bars = alt.hconcat(
    bar1,
    bar2
)

# (scatter | bars)
fig = alt.hconcat(scatter, bars).resolve_scale(
    color='independent', shape='independent',
    x='independent',
    y='independent'
).resolve_axis(
    x='independent',
    y='independent'
)


my_font = 'Helvetica'
fig = fig.configure_axis(
    titleFont=my_font,
    titleFontWeight='normal',
    labelFontSize=24,
    titleFontSize=28
).configure_header(
    titleFont=my_font,
    titleFontWeight='normal',
    labelFont=my_font,
    titleFontSize=28,
    labelFontSize=24
).configure_view(
    strokeWidth=0
).configure_title(
    font=my_font,
    fontSize=28
).configure_legend(
    titleFont=my_font,
    titleFontWeight='normal',
    labelFont=my_font,
    titleFontSize=28,
    labelFontSize=24,
    orient='right',
    titleLimit=0,
    labelLimit=0
)

# fig.save('figure_4.svg')
# fig.save('figure_4.png', scale_factor=3)
fig

/home/ahmad/anaconda3/envs/envbase/lib/python3.11/site-packages/altair/utils/core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/ahmad/anaconda3/envs/envbase/lib/python3.11/site-packages/altair/utils/core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/ahmad/anaconda3/envs/envbase/lib/python3.11/site-packages/altair/utils/core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False

alt.HConcatChart(...)